In [2]:
import os
import json
import shutil
import tempfile
import time
from typing import Tuple
import glob
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib

from monai.losses import DiceLoss, DiceCELoss

from monai.inferers import sliding_window_inference
from monai import transforms
from monai.networks.layers import Norm
from monai.transforms import (
    AsDiscrete,
    Activations,
    ScaleIntensityRanged,
    EnsureChannelFirstd,
    Orientationd,
    ScaleIntensityRange,
    RandCropByPosNegLabeld,
    Compose,
    RandRotate90d,
    SpatialCropd,
    KeepLargestConnectedComponent,
    Compose
)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.utils.enums import MetricReduction
from monai.networks.nets import SwinUNETR, UNet

from monai.data import decollate_batch, DataLoader, Dataset
from functools import partial

import torch
import torch.optim as optim
import torch.nn as nn

import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from sklearn.model_selection import train_test_split

import nibabel as nib
import numpy as np

# Visualizations
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
from IPython.display import HTML
sns.set_style("darkgrid")

In [18]:
files = glob.glob("out/withLargestConnected/*/*.nii.gz")

In [23]:
def show_irm(data: np.ndarray):
    # Create a figure with 3 subplots
    fig, axs = plt.subplots(1, 1)

    # Initialize images for each subplot
    im1 = axs.imshow(data[:, :, 0], cmap='gray', vmin=0, vmax=1)

    # Function to update the plots for each frame
    def update(frame):
        im1.set_data(data[:, :, frame])
        axs.set_title(f'Slice {frame}')
        axs.grid(False)


    # Create an animation
    num_slices = data.shape[2]
    anim = animation.FuncAnimation(fig, update, frames=num_slices, interval=100, blit=False)
    plt.close()
    return HTML(anim.to_html5_video())

In [43]:
data = nib.load(files[8]).get_fdata()

In [45]:
video = show_irm(data)

In [46]:
video

In [44]:
len(np.argwhere(data))

3955

In [47]:
h = np.argwhere(data)[0]
plt.imshow(data[:, :, h], cmap='gray', vmin=0, vmax=1)
# grid show false
plt.grid(False)
plt.show()

IndexError: index 317 is out of bounds for axis 2 with size 134

In [7]:
df = pd.read_csv("../PSCC_datachallenge/sub_filesLargestConnected.csv")

In [8]:
df

,id,rle,recist,volume,data_shape
0,LUNG1-001_vol_PredSeg,12180366 1 12180495 1 12180624 1 12180753 1 12...,30.084027,8.00,"(512, 512, 129)"
1,LUNG1-002_vol_PredSeg,3170838 1 3170972 1 3239445 2 3239579 1 3308053 1,1.381263,0.02,"(512, 512, 134)"
2,LUNG1-003_vol_PredSeg,28010917 3 28011039 4 28011161 6 28011283 6 28...,14.146931,1.63,"(512, 512, 123)"
3,LUNG1-004_vol_PredSeg,25500292 1 25500406 2 25500514 1 25555158 1 25...,8.789258,0.62,"(512, 512, 108)"
4,LUNG1-005_vol_PredSeg,15079576 1 15157912 1 15236248 1 15314584 1 15...,17.055126,0.38,"(512, 512, 153)"
...,...,...,...,...,...
95,LUNG1-096_vol_PredSeg,20198579 1 20266785 1 20266919 1 20267053 1 20...,56.649238,6.97,"(512, 512, 134)"
96,LUNG1-097_vol_PredSeg,10664148 2 10664259 2 10664370 2 10664480 3 10...,38.164072,22.85,"(512, 512, 111)"
97,LUNG1-098_vol_PredSeg,12870956 2 12935342 1 12935467 3 12999728 2 12...,9.003657,0.20,"(512, 512, 126)"
98,LUNG1-099_vol_PredSeg,20203018 1 20203151 2 20203286 2 20203419 3 20...,30.185230,11.51,"(512, 512, 134)"


In [9]:
df['id'] = df['id'].apply(lambda x: x[:9])

In [10]:
df["volume"][:50]

0      8.00
1      0.02
2      1.63
3      0.62
4      0.38
5      3.05
6     70.52
7      0.02
8      2.20
9      6.63
10     7.66
11    15.21
12     0.09
13     9.74
14     0.00
15    25.37
16     4.09
17     1.59
18    67.07
19     0.32
20     2.20
21     6.98
22     1.63
23    47.74
24     0.00
25     1.63
26    22.51
27    12.64
28     0.44
29     0.59
30     1.53
31     9.50
32    33.46
33     4.44
34     9.50
35    11.46
36    12.30
37    41.33
38    35.54
39    18.36
40     2.68
41    39.54
42    10.36
43    13.72
44     1.83
45    39.68
46     4.08
47     9.92
48     9.42
49     2.86
Name: volume, dtype: float64

In [11]:
# save
df.to_csv("sub_filesLargestConnected", index=False)